<a href="https://colab.research.google.com/github/GrigoryBartosh/hse07_nlp/blob/master/4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install transformers

In [1]:
import os
import re
import pandas as pd
import datetime

import numpy as np
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

from transformers import BertTokenizer, BertModel

from tqdm.auto import tqdm, trange
import matplotlib.pyplot as plt

import logging
logging.basicConfig(level=logging.CRITICAL)

PATH_LOGS = os.path.join('data', 'logs_tf')

PATH_DATASET = os.path.join('data', 'train_qa.csv')

PATH_DATASET_TEST = os.path.join('data', 'test.txt')
PATH_RESULTS = os.path.join('data', 'results.txt')

MAX_TEXT_LEN = 256

EPOCHS_1 = 1
EPOCHS_2 = 3
BATCH_SIZE = 16
LEARNING_RATE_1 = 0.0001
LEARNING_RATE_2 = 0.0001
W_L2_NORM = 0.0

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [2]:
dataset = pd.read_csv(PATH_DATASET)
dataset_texts = dataset['paragraph']
dataset_questions = dataset['question']
dataset_answers = dataset['answer']

In [3]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-multilingual-cased',
    do_lower_case=False
)

In [4]:
def prepare_sample(text, question, answer):
    answer = answer.lower()
    while (answer[0] == '.'):
        answer = answer[1:]
    while (answer[-1] in ['.', '?']):
        answer = answer[:-1]
        
    if answer not in text.lower():
        return [], []
    
    first = text.lower().find(answer)
    last = first + len(answer)
    
    text_1 = text[:first].strip()
    text_2 = text[first:last].strip()
    text_3 = text[last:].strip()
    text_tokens = tokenizer.tokenize(text_1)
    first = len(text_tokens)
    text_tokens += tokenizer.tokenize(text_2)
    last = len(text_tokens) - 1
    text_tokens += tokenizer.tokenize(text_3)
    
    question_tokens = tokenizer.tokenize(question)
    
    length = MAX_TEXT_LEN - len(question_tokens) - 3
    if len(text_tokens) > length:
        part_length = length // 3
        stride = 3 * part_length
        nrow = np.ceil(len(text_tokens) / part_length) - 2
        indexes = part_length * np.arange(nrow)[:, None] + np.arange(stride)
        indexes = indexes.astype(np.int32)

        max_index = indexes.max()
        diff = max_index + 1 - len(text_tokens)
        text_tokens += diff * [tokenizer.pad_token]

        text_tokens = np.array(text_tokens)[indexes].tolist()
        
        tokens = []
        labels = []
        for i, ts in enumerate(text_tokens):
            while ts[-1] == tokenizer.pad_token:
                ts = ts[:-1]
                
            tokens += [ts]
                
            lfirst = first - i * part_length
            llast = last - i * part_length
            
            mask = lfirst >= 0 and lfirst < len(ts) and llast >= 0 and llast < len(ts)
            labels += [((lfirst if mask else 0, mask), (llast if mask else 0, mask))]
    else:
        tokens = [text_tokens]
        labels = [((first, 1), (last, 1))]
        
    for i in range(len(tokens)):
        tokens[i] = [tokenizer.cls_token] + \
                    question_tokens + \
                    [tokenizer.sep_token] + \
                    tokens[i] + \
                    [tokenizer.sep_token]
        labels[i] = ((labels[i][0][0] + 2 + len(question_tokens), labels[i][0][1]),
                     (labels[i][1][0] + 2 + len(question_tokens), labels[i][1][1]))

    return tokens, labels

In [5]:
dataset_tokens, dataset_labels = [], []

for text, question, answer in tqdm(list(zip(dataset_texts, dataset_questions, dataset_answers))):
    tokens, labels = prepare_sample(text, question, answer)
    dataset_tokens += tokens
    dataset_labels += labels

x_train, x_val, y_train, y_val = train_test_split(dataset_tokens, dataset_labels, test_size=0.1)
train_data = list(zip(x_train, y_train))
val_data = list(zip(x_val, y_val))

In [6]:
def text_collate_fn(texts):
    max_len = max([len(text) for text in texts])
    masks = [[1] * len(text) + [0] * (max_len - len(text)) for text in texts]
    texts = [text + [tokenizer.pad_token] * (max_len - len(text)) for text in texts]
    texts = [tokenizer.convert_tokens_to_ids(text) for text in texts]
    texts = torch.LongTensor(texts)
    masks = torch.LongTensor(masks)

    return texts, masks

def collate_fn(data):
    texts, labels = zip(*data)

    texts, masks = text_collate_fn(texts)
    
    labels_first, labels_last = zip(*labels)
    labels_first_pos, labels_first_valid = zip(*labels_first)
    labels_last_pos, labels_last_valid = zip(*labels_last)
    
    labels_first_pos = torch.LongTensor(labels_first_pos)
    labels_first_mask = torch.LongTensor(labels_first_valid)
    labels_last_pos = torch.LongTensor(labels_last_pos)
    labels_last_mask = torch.LongTensor(labels_last_valid)
    
    return texts, masks, labels_first_pos, labels_first_mask, labels_last_pos, labels_last_mask

def infinit_data_loader(data_loader):
    while True:
        for x in data_loader:
            yield x

train_data_loader = data.DataLoader(
    dataset=train_data,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=8,
    collate_fn=collate_fn
)
val_data_loader = data.DataLoader(
    dataset=val_data,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=8,
    collate_fn=collate_fn
)

val_data_loader = infinit_data_loader(val_data_loader)

In [7]:
class TextClassifier(nn.Module):
    def __init__(self):
        super(TextClassifier, self).__init__()
        
        self.bert = BertModel.from_pretrained('bert-base-multilingual-cased')

        for param in self.bert.parameters():
            param.requires_grad = False

        layers = [
            nn.Linear(768, 128),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(128, 32),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(16, 2)
        ]
        self.layers = nn.Sequential(*layers)
        
    def masked_softmax(self, vec, mask, dim=1):
        masked_vec = vec * mask.float()
        max_vec = torch.max(masked_vec, dim=dim, keepdim=True)[0]
        exps = torch.exp(masked_vec - max_vec)
        masked_exps = exps * mask.float()
        masked_sums = masked_exps.sum(dim, keepdim=True)
        zeros = (masked_sums == 0)
        masked_sums += zeros.float()
        return masked_exps / masked_sums

    def forward(self, text, mask):
        x = self.bert(text, attention_mask=mask)[0]
        x = self.layers(x)
        x = self.masked_softmax(x, mask[:, :, None])
        return x

In [8]:
class MaskedLoss(nn.Module):
    EPS  = 1e-8

    def __init__(self):
        super(MaskedLoss, self).__init__()
        
    def forward(self, output, output_mask, target, target_mask):
        sm_0 = -torch.log(1 - output + MaskedLoss.EPS)
        sm_0_mask = output_mask * (1 - target_mask[:, None])
        sm_0 = sm_0 * sm_0_mask
        sm_0_mask = sm_0_mask.sum(dim=1)
        sm_0 = sm_0.sum(dim=1) / torch.max(sm_0_mask, torch.ones_like(sm_0_mask))
        
        sm_1 = torch.gather(output, 1, target[:, None]).squeeze()
        sm_1 = -torch.log(sm_1 + MaskedLoss.EPS)
        sm_1 = sm_1 * target_mask
        
        one = torch.ones_like(target_mask.sum())
        loss = sm_0.sum() / torch.max((1 - target_mask).sum(), one) + \
               sm_1.sum() / torch.max(target_mask.sum(), one)
        
        return loss

In [9]:
def make_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def get_summary_writer():
    name = str(datetime.datetime.now())[:19]
    make_dir(PATH_LOGS)
    logs_path = os.path.join(PATH_LOGS, name)
    return SummaryWriter(logs_path)

In [10]:
def train(model, criterion, optimizer, scheduler, epochs):
    summary_writer = get_summary_writer()
    step = 0
    last_val = 0
    
    for epoch in trange(epochs):
        model.train()
        for texts, masks, lfp, lfm, llp, llm in train_data_loader:
            texts = texts.to(device)
            masks = masks.to(device)
            lfp = lfp.to(device)
            lfm = lfm.to(device)
            llp = llp.to(device)
            llm = llm.to(device)
            
            optimizer.zero_grad()

            ps = model(texts, masks)
            loss = criterion(ps[:, :, 0], masks, lfp, lfm) + \
                   criterion(ps[:, :, 1], masks, llp, llm)

            loss.backward()
            optimizer.step()

            step += len(texts)
            last_val += len(texts)
            summary_writer.add_scalar('Train/loss', loss.item(), step)
            if last_val >= 10 * BATCH_SIZE:
                model.eval()
                with torch.no_grad():
                    texts, masks, lfp, lfm, llp, llm = next(val_data_loader)
                    texts = texts.to(device)
                    masks = masks.to(device)
                    lfp = lfp.to(device)
                    lfm = lfm.to(device)
                    llp = llp.to(device)
                    llm = llm.to(device)

                    ps = model(texts, masks)
                    loss = criterion(ps[:, :, 0], masks, lfp, lfm) + \
                           criterion(ps[:, :, 1], masks, llp, llm)

                    summary_writer.add_scalar('Validation/loss', loss.item(), step)
                    
                model.train()
                last_val = 0
                
        if scheduler:
            scheduler.step()

In [11]:
model = TextClassifier()
model.to(device)

criterion = MaskedLoss()

optimizer = optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    LEARNING_RATE_1,
    weight_decay=W_L2_NORM
)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.3)

train(model, criterion, optimizer, scheduler, EPOCHS_1)

In [12]:
#train(model, criterion, optimizer, 4)

In [13]:
for param in model.parameters():
    param.requires_grad = True

optimizer = optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    LEARNING_RATE_2,
    weight_decay=W_L2_NORM
)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.3)

train(model, criterion, optimizer, scheduler, EPOCHS_2)

In [ ]:
#optimizer = optim.Adam(
#    filter(lambda p: p.requires_grad, model.parameters()),
#    0.00001,
#    weight_decay=W_L2_NORM
#)

#train(model, criterion, optimizer, scheduler, 1)

In [41]:
def get_best(ps):
    n = len(ps)
    first, last, mx = 0, 0, 0
    for i in range(n):
        for j in range(i, n):
            lmx = ps[i, 0] * ps[j, 1]
            if mx < lmx:
                mx, first,last = lmx, i, j
                
    return first, last, mx

def make_answer(tokens):
    s = ''
    for token in tokens:
        if token == tokenizer.unk_token:
            continue

        if token[0] == '#':
            s += token.replace('#', '')
        else:
            s += ' ' + token
        
    t = ''
    for i in range(len(s) - 1):
        if 
            
    return s.strip()
                

with open(PATH_DATASET_TEST, 'r') as file:
    dataset_test = file.readlines()[1:]
    
res = []
model.eval()
with torch.no_grad():
    for sample in tqdm(dataset_test):
        _, question_id, paragraph, question = sample.split('\t')

        question_tokens = tokenizer.tokenize(question)
        text_tokens = tokenizer.tokenize(paragraph)
        
        all_tokens = [tokenizer.cls_token] + \
                     question_tokens + \
                     [tokenizer.sep_token] + \
                     text_tokens + \
                     [tokenizer.sep_token]

        length = MAX_TEXT_LEN - len(question_tokens) - 3
        if (len(text_tokens) > length):
            part_length = length // 3
            stride = 3 * part_length
            nrow = np.ceil(len(text_tokens) / part_length) - 2
            indexes = part_length * np.arange(nrow)[:, None] + np.arange(stride)
            indexes = indexes.astype(np.int32)

            max_index = indexes.max()
            diff = max_index + 1 - len(text_tokens)
            text_tokens += diff * [tokenizer.pad_token]

            text_tokens = np.array(text_tokens)[indexes].tolist()

            first, last, mx = 0, 0, 0
            for i, ts in enumerate(text_tokens):
                while ts[-1] == tokenizer.pad_token:
                    ts = ts[:-1]

                ts = [tokenizer.cls_token] + \
                     question_tokens + \
                     [tokenizer.sep_token] + \
                     ts + \
                     [tokenizer.sep_token]

                texts, masks = text_collate_fn([ts])
                texts = texts.to(device)
                masks = masks.to(device)

                lps = model(texts, masks)[0]
                lps = lps[2 + len(question_tokens):]
                lps[:, 0] = F.softmax(lps[:, 0], 0)
                lps[:, 1] = F.softmax(lps[:, 1], 0)
                lps = lps.cpu().numpy()
                
                lfirst, llast, lmx = get_best(lps)
                if mx < lmx:
                    mx = lmx
                    first = lfirst + i * part_length
                    last = llast + i * part_length
                
            first += 2 + len(question_tokens)
            last += 2 + len(question_tokens)
                    
        else:
            texts, masks = text_collate_fn([all_tokens])
            texts = texts.to(device)
            masks = masks.to(device)

            ps = model(texts, masks)[0]
            ps[:, 0] = F.softmax(ps[:, 0], 0)
            ps[:, 1] = F.softmax(ps[:, 1], 0)
            ps = ps.cpu().numpy()
            
            first, last, _ = get_best(ps)
            
        s = make_answer(all_tokens[first:last + 1])
                
        ans = dp([paragraph], [question])[0][0]
        print(s)
        print(ans)
        print()
            
        res += [question_id + '\t' + s]
        
res = '\n'.join(res)
with open(PATH_RESULTS, 'w') as file:
    file.write(res)

зарплата равна стоимости рабочей силы
не упоминается

в определенном принятом диапазоне колебаний
в определенном принятом диапазоне колебаний

18 млрд рублей
18 млрд рублей

новые графические чипы с поддержкой аппаратного ускорения формирования 3D - графики
чипы YGV611 и YGV612

с группой Автограф
Автограф

лимфатическая
лимфатическая

Гаспаро да Сало ( ум . 1609 ) из города Бреши и Андреа Амати
Андреа Амати

Аскетический образ жизни и практика совершенствования своих духовных качеств
Аскетический образ жизни и практика совершенствования своих духовных качеств

Великобритания , Канада
Великобритания, Канада

северо - западные иранские диалекты
северо-западные иранские диалекты

как хоровая картина , и как групповой портрет
как хоровая картина , и как групповой портрет

одна треть
одна треть

денег у него хватило лишь на два года обучения
чтобы сын ходил в школу

на сторону маньчжуров
на сторону маньчжуров

Эмиля Берлинера
американский инженер еврейского происхождения Эмиль Берлинер

28

семь замен
семь

тем , что нижние парциальные соцветия у них развиты и ветвятся гораздо сильнее верхних
обильным ветвлением

жюстокор
жюстокор

английским инженером Аланом Блюмлейном
английским инженером Аланом Блюмлейном

композиторами , авторами песен и авторами музыкальных пьес
композиторами, авторами песен и авторами

16 МБ
16 МБ

незаменимым руководством для врачей - хирургов
незаменимым руководством для врачей-хирургов

68 - ое место
68-ое место

не более 8 %
не более 8 %

Иридий
Иридий

Мухаммед вновь казнил одного из участников следующего монгольского посольства
отказался идти на сделку с неверными

В 1940 году
В 1940 году

международное рейтинговое агентство Standard & Poor s
международное рейтинговое агентство Standard & Poor’s

в сквере по улице Загорской
в сквере по улице Загорской

на острове Родос
на острове Родос

предоставление и ограничение прав должны официально фиксироваться этим лицом , которое несет ответственность за сохранность официальных записей , обеспечение и

взаимоотношения орхидеи Chiloglottis trapeziformis и осы Neozeleboria cryptoides
взаимоотношения орхидеи Chiloglottis trapeziformis и осы Neozeleboria cryptoides

сокращение расходов и расширирение своего влияния в немецкоязычной и франкоязычной частях Швейцарии
сокращение расходов и расширирение своего влияния

для ежедневного управления промежуточной целью
для ежедневного управления промежуточной целью

носителю просторечия
носителю просторечия

в относительно небольших
в относительно небольших

на основе антиквы с диакритическими значками
в литературном языке и большинстве говоров совпал с /r/

перестройки
перестройки

переключалась на третью передачу
сразу переключалась на третью передачу

томные или игривые любовные песни
томные или игривые любовные песни

пять региональных банков
пять

не более 1 % жителей
не более 1 %

эстетическое выражение социалистически осознанной концепции мира и человека
эстетическое выражение социалистически осознанной концепции мира и человека

После поя

дополнительные переменные в ответ на запросы покупателей
дополнительные переменные в ответ на запросы покупателей

в экспедициях
в экспедициях

Таймырская
Таймырская

Нижнесаксонский высший земельный суд
Нижнесаксонский высший земельный суд

Надписи на иконах
В греческих

Павсаний
Павсаний

Иордан
Иордан

Оптика
Оптика

в Фрисландии
в Фрисландии

В 60 - х гг . до н . э
В 60-х гг. до н. э.

архитектурный облик
архитектурный облик

религиозные
религиозные

в Москве
в Москве

от 3 , 5 до 6 лет
от 3,5 до 6 лет

Блуждающий полузащитник
Блуждающий полузащитник

Роберта Ремака ( 1852 ) и Рудольфа Вирхова ( 1855 )
Роберта Ремака

с кровью
с румянцем, женскими щеками

3 ноября 1881 г
4 июня 1880 г.

в военизированной охране МПС СССР
в военизированной охране МПС СССР

в течение трёх лет
в течение трёх лет

расчётная норма прибыли
расчётная норма прибыли

21 мая 1590 года
21 мая 1590 года

в 1632 году
в 1632 году

на учение Аристотеля и изучение Талмуда
на учение Аристотеля и изучение Талмуда

с 

обеспечить себя доказательствами существования объектов авторских прав на определённую дату
все перечисленные механизмы не обеспечивают доказательства авторства

Среднее общеобразовательное учебное заведение
Среднее общеобразовательное учебное заведение

Тревитик
Ричард Тревитик

проявления дорожек и разрезания магнитной ленты
разрезания и склейки магнитной ленты

работу
два десятилетия

Комсомольск - на - Амуре
Комсомольск-на-Амуре

4 маршрута
8

Недоговорённость и иронический нюанс
Недоговорённость и иронический нюанс

27 - я
22-я

март 1875
В апреле 1873 года

1913 год
в 1915 году

11 , 2 км
11,2 км трассы, 13 станций и 12 составов

технология структурного проектирования программ
технология структурного проектирования программ

густого и влажного леса
густого и влажного

в XVIII веке
в XVIII веке

Волновой принцип
Волновой принцип

GNU Hurd
GNU Hurd

постолы
одна на одну, плащи

к V VI вв
В конце Ахеменидской эпохи

Баутцен
Баутцен

в своих работах Система общества и Система природы

Никео - Цареградский Символ веры
Никео-Цареградский Символ веры

1809 - 49
1809-49

Тё Тикун
Тё Тикун

450 тыс . особей
450 тыс. особей

Хайрама Сибли
Хайрама Сибли

Цзинь
Цзинь

итальянский , португальский , английский , немецкий и французский
итальянский, португальский, английский, немецкий и французский

родной язык самый употребляемый
родной язык самый употребляемый

авиация и морской флот
авиация и морской флот

немая школа
немая школа

чтобы узнать состав и свойства незнакомых веществ
чтобы узнать состав и свойства незнакомых веществ

продав убыточный бизнес под видом прибыльного
продав убыточный бизнес под видом прибыльного

186 организаций
186 организаций

The Who , Yardbirds , Hollies
The Who, Yardbirds, Hollies

к труднопрорастающим
к труднопрорастающим

China UnionPay в Китае . Там она внедрялась в течение нескольких лет , выпустив указание о проведение всех транзакции внутри страны через национальную карточную систему . Теперь карты China UnionPay принимаются в 26 странах м

сверхрпаразиты 3 - го ( третичные ) и 4 - го порядка
сверхрпаразиты 3-го (третичные) и 4-го порядка

Нижнесаксонский Ландтаг
Нижнесаксонский Ландтаг

справедливой конкуренции
справедливой конкуренции

на живичном скипидаре и скипидарном масле
на живичном скипидаре и скипидарном масле

ОАО Мосстройпластмасс
ОАО Мосстройпластмасс

целое число
0

Университет Палацкого
Университет Палацкого

В 1923 году
1902

* i , * ī , * u , * ū , * a , * ā
гласные *i, *ī, *u, *ū, *a, *ā

В 1917 году
В 1917 году

в Республике Тыва
Всё поголовье оленей страны

простофиля
простофиля

с Тирренским морем
с Тирренским

новое здание городской ратуши
новое здание городской ратуши

аграф
брасьер

на лбу
на пороге

большие количества легирующих добавок
большие количества легирующих добавок

славянский анклав
славянский анклав

за заем
за заем

притягиваются друг к другу , на меньших отталкиваются
притягиваются друг к другу

Уныние , 1849
Уныние

тридентский обряд
тридентский

в состав какого - либо семейства
како

In [37]:
from deeppavlov import build_model, configs

dp = build_model(configs.squad.squad_ru, download=True)

2019-12-07 15:37:50.318 INFO in 'deeppavlov.core.data.utils'['utils'] at line 80: Downloading from http://files.deeppavlov.ai/deeppavlov_data/squad_model_ru_1.4_cpu_compatible.tar.gz to /home/grigory/.deeppavlov/squad_model_ru_1.4_cpu_compatible.tar.gz
INFO:deeppavlov.core.data.utils:Downloading from http://files.deeppavlov.ai/deeppavlov_data/squad_model_ru_1.4_cpu_compatible.tar.gz to /home/grigory/.deeppavlov/squad_model_ru_1.4_cpu_compatible.tar.gz
100%|██████████| 533M/533M [00:39<00:00, 13.6MB/s] 
2019-12-07 15:38:29.629 INFO in 'deeppavlov.core.data.utils'['utils'] at line 237: Extracting /home/grigory/.deeppavlov/squad_model_ru_1.4_cpu_compatible.tar.gz archive into /home/grigory/.deeppavlov/models
INFO:deeppavlov.core.data.utils:Extracting /home/grigory/.deeppavlov/squad_model_ru_1.4_cpu_compatible.tar.gz archive into /home/grigory/.deeppavlov/models
2019-12-07 15:38:43.898 INFO in 'deeppavlov.core.data.utils'['utils'] at line 80: Downloading from http://files.deeppavlov.ai/emb

2019-12-07 15:45:20.561 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 615: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleGRUCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnGRUCell later. 
INFO:deeppavlov.core.layers.tf_layers:
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleGRUCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnGRUCell later. 
2019-12-07 15:45:20.668 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 615: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleGRUCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnGRUCell later. 
INFO:deeppavlov.core.layers.tf_layers:
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleGRUCell is used. It is okay for inference mode, but if you train your model w

2019-12-07 15:45:32.86 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /home/grigory/.deeppavlov/models/squad_model_ru/model]
INFO:deeppavlov.core.models.tf_model:[loading model from /home/grigory/.deeppavlov/models/squad_model_ru/model]


In [38]:
with open(PATH_DATASET_TEST, 'r') as file:
    dataset_test = file.readlines()[1:]
    
res = []
for sample in tqdm(dataset_test):
    _, question_id, paragraph, question = sample.split('\t')
    ans = dp([paragraph], [question])[0][0]
    
    res += [question_id + '\t' + ans]
        
res = '\n'.join(res)
with open(PATH_RESULTS, 'w') as file:
    file.write(res)